<p style="text-align:center">
    <a href="https://www.ict.mahidol.ac.th/en/" target="_blank">
    <img src="https://www3.ict.mahidol.ac.th/ICTSurveysV2/Content/image/MUICT2.png" width="400" alt="Faculty of ICT">
    </a>
</p>

# Lab08: ML Basics: Classification - Tutorial


This tutorial will provide hands-on practice in:
- building classifier models for binary-class and multi-class classification.
- collecting evaluation metrics
- using cross-validation to improve reliability of training-set / validation-set results.
- using various types of classifier algorithms
- approaches to dealing with imbalanced dataset problems.


## Library of Helper Functions

- This `Library of Helper Functions` is designed to help simplify many tasks when evaluating models on different datasets. Make use of these functions to save your time in trialling different models.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import learning_curve
import warnings
from sklearn.exceptions import FitFailedWarning

def _show_classification_report(model, y_true, y_pred, target_names):
    '''
        Function to print performance metrics
    '''
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, pos_label=1, average='weighted')
    specificity = recall_score(y_true, y_pred, pos_label=0, average='weighted')
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("Classification Report:")
    class_report = classification_report(y_true, y_pred, target_names=target_names)
    print(class_report)
    res = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
    return pd.json_normalize(res, sep='_')
    
def _show_confusion_matrix(model, y_true, y_pred, target_names):
    '''
        Function to plot confusion matrix
    '''
    cm = confusion_matrix(y_true, y_pred, labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=target_names)
    disp.plot(cmap=plt.cm.Blues,)
    plt.gcf().set_size_inches(3.5, 3.5)
    disp.ax_.set_title(f'Confusion Matrix for {model.__class__.__name__}', fontsize=8)
    plt.show()
    
def _plot_histogram_of_frequencies(data, ax=None):
    if not ax:
        fig, ax = plt.subplots(figsize=(7,2.5))
    unique_values, counts = np.unique(data, return_counts=True)
    barh = plt.bar(unique_values, counts)
    plt.xlabel("Values")
    plt.ylabel("Frequency")
    plt.title("Histogram")
    plt.xticks(unique_values)
    ax.bar_label(barh, fmt='%.2f')
    ax.set_ylim(bottom=0, top=1.25*max(counts))
    print('Class Split:', counts/sum(counts))
    plt.show()
    
def _make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10):
    def _plot_learning_curve(model, train_sizes, train_scores, valid_scores, metric='F1 Score', plt_text='', ax=None):
        if not ax:
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 3), sharey=True, sharex=True)
        train_errors = train_scores.mean(axis=1)
        valid_errors = valid_scores.mean(axis=1)
        ax.plot(train_sizes, train_errors, "r-+", linewidth=2, label="train")
        ax.plot(train_sizes, valid_errors, "b-", linewidth=3, label="valid")
        ax.set_xlabel("Training set size")
        ax.set_ylabel(f'{metric}')
        # plt.gca().set_xscale("log", nonpositive='clip')
        ax.grid()
        ax.legend(loc="upper right")
        ax.set_ylim(bottom=0, top=1.25*max([1]))
        ax.set_title(f'{model.__class__.__name__}\n{plt_text}', fontsize=8)
        plt.show()
        
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FitFailedWarning)
        train_sizes, train_scores, valid_scores = learning_curve( model, 
                                                    X_train, y_train, 
                                                    train_sizes=np.linspace(0.01, 1.0, num_training_sizes), # e.g. `num` size intervals, from 1% to 100%
                                                    cv=5,     # CV=5 means  Train = 80%  , Test = 20%.
                                                              # CV=10 means Train = 90%  , Test = 10%.
                                                              #   - The fit/predict is repeated 5 times with random samples taken from X/Y.
                                                              #   - The resulting error is the average across all 5 trials; so a smoother and fairer result than CV=1 , which is hold-out.
                                                    scoring=scoring,
                                                    n_jobs=-1
                                                )
    _plot_learning_curve(model, train_sizes, train_scores, valid_scores, metric=scoring.replace('_',' ').title(), plt_text='')


# Part 1 - Templates for Classification Models


### Binary Classification Model Example:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                   n_clusters_per_class=len(target_names)
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

model = LogisticRegression()
model.fit( X_train, y_train )
y_pred = model.predict( X_test )
# print(y_pred)

### Multi-Class Classification Model Example:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B','C']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                   n_clusters_per_class=len(target_names)
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

model = LogisticRegression()
model.fit( X_train, y_train )
y_pred = model.predict( X_test )
# print(y_pred)

# Part 2 - Templates for Classification Models with Metric Evaluation + Learning Curves


### Binary Classification Model Example:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                   n_clusters_per_class=len(target_names)
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

model = LogisticRegression()
model.fit( X_train, y_train )
y_pred = model.predict( X_test )

_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)

### Multi-Class Classification Model Example:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B','C']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                   n_clusters_per_class=len(target_names)
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

model = LogisticRegression()
model.fit( X_train, y_train )
y_pred = model.predict( X_test )

_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)

# Part 3 - Upgrading Hold-out to Cross Validation:

* Example K Folds:

In [ ]:
from sklearn.model_selection import train_test_split, KFold, \
                                    StratifiedKFold, \
                                    StratifiedGroupKFold, \
                                    cross_val_score

# Shuffled K-Folds:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_train, y_train, 
                         cv=kf, 
                         scoring='f1_weighted')

# Shuffled Stratified K-Folds by Class:
skf = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_val_score(model, X_train, y_train, 
                         cv=skf, 
                         scoring='f1_weighted')

# # Shuffled Stratified K-Folds by Group:
groups = [ 1,1,2,2,2,... ] # Group per X_train record.
groups = [0]*len(X_train)
gkf = StratifiedGroupKFold(n_splits=5, shuffle=True)
# scores = cross_val_score(model, X_train, y_train, 
#                          groups=groups, cv=gkf, 
#                          scoring='f1_weighted')

* Example implementation code:

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, \
                                    StratifiedKFold, \
                                    GroupKFold, \
                                    cross_val_score
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                   n_clusters_per_class=len(target_names)
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

model = LogisticRegression()

scoring='f1_weighted'

# K-Fold Cross-Validation on the Training Set:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
# Perform cross validation on the training data:
train_f1_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=scoring)

print( f'Cross Validation {scoring} Scores: {train_f1_scores}')
print(f'Mean: {np.mean(train_f1_scores):.4f}')
print(f'Std:  {np.std(train_f1_scores):.4f}')


# Proceed to fit on full training set & evaluate on test set:
model.fit( X_train, y_train )
y_pred = model.predict( X_test )

_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring=scoring, num_training_sizes=10)

# Part 4 - Many Classifier Algorithms:

#### LogisticRegression Model:

✅ **Logistic Regression** provides **probabilistic predictions** and performs well when features are correlated. **Logistic Regression often performs better**, making it a preferred choice for structured numerical datasets like loan approvals.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

#### Naive Bayes Model:

✅ **Naïve Bayes** is a simple, fast, and computationally efficient classifier, especially effective for text-based problems. However, its **independence assumption** means that each feature does not influence another feature; and this may not always hold in real-world data, which can impact accuracy.
- Despite Naïve Bayes' limitations, its **mathematical simplicity** and **interpretability** make it a strong choice in many applications, including **spam detection** and **text classification**.

In [ ]:
# from sklearn.naive_bayes import GaussianNB

# model = GaussianNB()

#### Linear (Stochastic Gradient Descent) Classifier Model:

✅ **SGDClassifier** works on a similar premise as **LogisticRegression**, however it allows for partial fitting to iteratively control its learning behaviour (sample-by-sample) and enabling scalability on large datasets. This is unavailable in LogisticRegression, which learns by batch. As SGD requires careful tuning, standard LogisticRegression is often preferred.

In [ ]:
# from sklearn.linear_model import SGDClassifier

# model = SGDClassifier(loss="log_loss", random_state=0)

#### DecisionTree Model:

✅ Decision Trees are highly interpretable classifiers, capable of capturing non-linear relationships and handling both categorical and numerical data. However, they are susceptible to overfitting, particularly with complex trees, and can create unstable models if the training data is slightly altered.

Decision Trees' intuitive visualization and ability to provide explicit decision rules make them valuable for tasks like feature importance analysis and applications where understanding the decision-making process is crucial, such as medical diagnosis and risk assessment.

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# model = DecisionTreeClassifier()

#### RandomForest Model:

✅ Random Forests are `ensemble` classifiers (consisting of many Decision Trees). They often give high accuracy and even on medium-complexity datasets. However, they can be less interpretable than single decision trees and do require more computational resources for training and prediction, especially with a large number of trees. To start, we will use 100 trees (`n_estimators=100`) and all available CPU cores to train them (`n_jobs=-1`).

Random Forests are more resistance to overfitting and have an ability to provide feature importance estimates. They are useful to test and you can often expect high accuracy and overall generalized fits.

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(n_estimators=100, n_jobs=-1)

#### Support Vector Machine Model:
✅ Support Vector Machines (SVMs) are `sophisticated` classifiers, for linear and `non-linear` data problems by using their transformation (`kernel`) function. 

Although computationally expensive for large datasets, they usually give a `generalized fit` and can be very accurate if you can **find the right transformation function** (kernel, parameters and regularization parameter), which can be challenging.

In [ ]:
# from sklearn.svm import SVC

# model = SVC()

# Part 5 - Experimenting with many models:


In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    random_state=0
                                   )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight

def get_models(): 
    class_weights = dict(zip(np.unique(y_train), 
    compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)     ))
    return [   
        LogisticRegression( class_weight='balanced', solver='lbfgs' ),
        # SGDClassifier(loss="log_loss", random_state=0, class_weight=class_weights ),
        # GaussianNB(),
        # DecisionTreeClassifier(class_weight='balanced' ),
        # RandomForestClassifier(n_estimators=100, class_weight='balanced_subsample', n_jobs=-1),
        # SVC( C=1.0, kernel='rbf', degree=3, gamma='scale', class_weight='balanced' ),
        # SVC( C=1.0, kernel='linear', class_weight='balanced' ),
        ]

In [ ]:
models = get_models()
for model in models:
    print(model.__class__.__name__)
    model.fit( X_train, y_train )
    y_pred = model.predict( X_test )
    _show_confusion_matrix(model, y_test, y_pred, target_names)
    _show_classification_report(model, y_test, y_pred, target_names)
    _make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)

# Part 6 - Templates for Classification Models with Imbalanced Datasets:


## `Check Imbalance:`

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)

_plot_histogram_of_frequencies(y_train)

## Use `Stratify by y`

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0, stratify=y)

_plot_histogram_of_frequencies(y_train)

## Use `class_weight='balanced'`

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)
_plot_histogram_of_frequencies(y_train)


# Cost-Sensitive Logistic Regression Using class_weight='balanced'
# Initialize Logistic Regression with class_weight='balanced'
model = LogisticRegression(solver='liblinear', 
                           class_weight='balanced', 
                           random_state=0)


model.fit( X_train, y_train )
y_pred = model.predict( X_test )
_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)





## Use `Undersampling`

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)
_plot_histogram_of_frequencies(y_train)

In [ ]:
import pandas as pd

def get_undersampled( X_train, y_train, target_column_name='target' ):
    X_train_cp = pd.DataFrame(X_train.copy())
    X_train_cp[target_column_name] = y_train
    y_0 = X_train_cp[X_train_cp[target_column_name] == 0]
    y_1 = X_train_cp[X_train_cp[target_column_name] == 1]
    y_0_undersample = y_0.sample(y_1.shape[0])
    df_train_undersampled = pd.concat([y_0_undersample, y_1], axis = 0)
    return df_train_undersampled

target_name = 'target'
df_train_undersampled = get_undersampled( X_train, y_train, target_column_name=target_name )
X_train_under = df_train_undersampled.drop(columns=[target_name])
y_train_under = df_train_undersampled[target_name]

_plot_histogram_of_frequencies( y_train_under )

In [ ]:
model.fit( X_train_under, y_train_under )
y_pred = model.predict( X_test )
_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)

## Use `Oversampling`

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)
_plot_histogram_of_frequencies(y_train)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SVMSMOTE

print('Original dataset shape %s' % Counter(y_train))
sm = SVMSMOTE(random_state=42)
X_train_SMOTE, y_train_SMOTE = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_SMOTE))
_plot_histogram_of_frequencies( y_train_SMOTE )

In [ ]:
model.fit( X_train_SMOTE, y_train_SMOTE )
y_pred = model.predict( X_test )
_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)

## Use `Curriculum Learning`
**Theory:**

Curriculum learning for imbalanced datasets involves training a model progressively, starting with simpler training data representations and moving to more difficulty samples to improve model performance. In theory, the model is first exposed to simplified, balanced versions of the data. As the model progresses, the data's complexity is gradually increased, introducing more realistic imbalances and challenging cases. This staged approach allows the model to learn fundamental patterns without being overwhelmed by the initial imbalance. By focusing on easier examples first, the model develops a robust understanding of the minority class. This hopefully leads to better generalization and improved performance on the imbalanced dataset.

This original method was inspired by the general concept of curriculum learning and published here:
> Reference:  **Yoshua Bengio et al. on curriculum learning is foundational: Bengio, Y., Louradour, J., Collobert, R., & Weston, J. (2009). Curriculum learning. Proceedings of the 26th annual international conference on machine learning, 1  41-48. [https://dl.acm.org/doi/10.1145/1553374.1553380](https://dl.acm.org/doi/10.1145/1553374.1553380)**

**Implementation:**

In our implementation below requires the use of the `model.partial_fit( ... )` method. Not all classifiers have this, so we can try two:


| Model Type         | Model Class                      | Supports `partial_fit`? |
| ------------------ | -------------------------------- | ----------------------- |
| **Linear Models**  | `SGDClassifier`                  | ✅ Yes                  |
| **Naive Bayes**    | `GaussianNB`                     | ✅ Yes                  |

**Steps:**

1. We use an **"easy" dataset** created through `undersampling the majority class`, providing a balanced, albeit reduced, training set. This allows the model to initially learn fundamental patterns without the bias introduced by severe class imbalance.
2. Subsequently, a **"medium" dataset**, generated using `SMOTE-SVM oversampling`, introduces synthetic minority class instances, refining the model's ability to recognize these crucial patterns.
3. The **"hard" dataset** is the **original imbalanced data**, used for fine-tuning from the prior training rounds. This staged approach aims to improve model generalization and performance on the minority class.    |

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

target_names = ['A','B']
X, y = datasets.make_classification(n_samples=1000, 
                                    n_informative=4,
                                    n_features=6, 
                                    n_classes=len(target_names),
                                    n_clusters_per_class=len(target_names), 
                                    weights=[0.75,0.25],
                                    random_state=0
                                   )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=0)
_plot_histogram_of_frequencies(y_train)

* Fit model on imbalanced data as baseline:

In [ ]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss="log_loss", random_state=0)

model.fit( X_train, y_train )
y_pred = model.predict( X_test )

_show_confusion_matrix(model, y_test, y_pred, target_names)
_show_classification_report(model, y_test, y_pred, target_names)
_make_learning_curve(model, X_train, y_train, scoring="f1_weighted", num_training_sizes=10)


In [ ]:
from sklearn.preprocessing import StandardScaler

def curriculum_learning_partial_fit( model, X_train, y_train, X_test, y_test,
                                    xy_train=[(X_train_under, y_train_under),
                                              (X_train_SMOTE, y_train_SMOTE),
                                              (X_train, y_train)], 
                                    complexity_levels=[0.2, 0.5, 1.0] ):
    scaler = StandardScaler()
    _ = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    for (_x, _y) in xy_train:
        _x_scaled = scaler.transform(_x)
        for level in complexity_levels:
            subset_size = int(level * len(_x))
            X_curr, y_curr = _x_scaled[:subset_size], _y[:subset_size]
            try:
                model.partial_fit(X_curr, y_curr, classes=np.unique(_y))
            except Exception as e:
                print(model.__class__.__name__, 'was unable to call .partial_fit( .. ).\nAborting.' )
                return None
    
    y_pred = model.predict(X_test_scaled)
    _show_classification_report(model, y_test, y_pred, target_names)
    _show_confusion_matrix(model, y_test, y_pred, target_names)

In [ ]:

curriculum_learning_partial_fit( model, X_train, y_train, X_test, y_test,
                                xy_train=[(X_train_under, y_train_under),
                                          (X_train_SMOTE, y_train_SMOTE),
                                          (X_train, y_train)], 
                                complexity_levels=[0.2, 0.5, 1.0] )
